In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import warnings

In [ ]:
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

np.random.seed(42)

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
CSV_FILE_PATH = '../scraper/data/crypto_data.csv'

print("Loading data from CSV...")
df = pd.read_csv(CSV_FILE_PATH)

print(f"\nData loaded successfully!")
print(f"Total records: {len(df)}")
print(f"Shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
print("="*80)
print("INITIAL DATA INSPECTION")
print("="*80)

# Basic info
print("\n--- Dataset Info ---")
print(df.info())

print("\n--- Data Types ---")
print(df.dtypes)

print("\n--- Dataset Shape ---")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

print("\n--- Column Names ---")
print(df.columns.tolist())

print("\n--- Statistical Summary ---")
print(df.describe())

print("\n--- Missing Values Count ---")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])
print(f"\nTotal missing values: {df.isnull().sum().sum()}")

print("\n--- Duplicate Rows ---")
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

print("\n--- Unique Values per Column ---")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

In [ ]:
print("\n" + "="*80)
print("DATA CLEANING")
print("="*80)

# Create a copy for cleaning
df_clean = df.copy()

# 1. Convert timestamp to datetime
print("\n1. Converting timestamp to datetime...")
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])
df_clean['date'] = df_clean['timestamp'].dt.date
df_clean['hour'] = df_clean['timestamp'].dt.hour
df_clean['day_of_week'] = df_clean['timestamp'].dt.dayofweek
df_clean['day_name'] = df_clean['timestamp'].dt.day_name()
print("Timestamp converted")

# 2. Handle zero/negative values in price columns
print("\n2. Checking for zero/negative prices...")
price_cols = ['price_usd', 'volume_24h', 'market_cap']
for col in price_cols:
    if col in df_clean.columns:
        zero_count = (df_clean[col] == 0).sum()
        neg_count = (df_clean[col] < 0).sum()
        print(f"   {col}: {zero_count} zeros, {neg_count} negative values")
        # Replace zeros with NaN for proper handling
        df_clean.loc[df_clean[col] == 0, col] = np.nan

# 3. Remove duplicates
print("\n3. Removing duplicates...")
before_dup = len(df_clean)
df_clean = df_clean.drop_duplicates()
after_dup = len(df_clean)
print(f"   Removed {before_dup - after_dup} duplicate rows")

# 4. Sort by timestamp and token
print("\n4. Sorting data by timestamp and token...")
df_clean = df_clean.sort_values(['token_symbol', 'timestamp']).reset_index(drop=True)
print("Data sorted")

# 5. Check data quality by source
print("\n5. Data quality by source:")
for source in df_clean['source'].unique():
    source_data = df_clean[df_clean['source'] == source]
    print(f"\n   {source}:")
    print(f"   - Records: {len(source_data)}")
    print(f"   - Price completeness: {(1 - source_data['price_usd'].isnull().sum()/len(source_data))*100:.2f}%")
    print(f"   - Tokens covered: {source_data['token_symbol'].nunique()}")

print(f"\nCleaned dataset shape: {df_clean.shape}")

In [ ]:
print("\n" + "="*80)
print("DATA INTEGRATION - MERGING MULTIPLE SOURCES")
print("="*80)

# Strategy: For each timestamp + token, average the values from different sources
print("\n1. Creating integrated dataset with averaged values...")

# Group by timestamp and token, take mean of numeric columns
numeric_cols = ['price_usd', 'volume_24h', 'market_cap', 'price_change_1h', 
                'price_change_24h', 'price_change_7d', 'circulating_supply']

# Create aggregation dictionary
agg_dict = {col: 'mean' for col in numeric_cols if col in df_clean.columns}
agg_dict['source'] = lambda x: ', '.join(x.unique())  # Combine source names
agg_dict['token_name'] = 'first'

# Round timestamp to nearest 5 minutes for better grouping
df_clean['timestamp_rounded'] = df_clean['timestamp'].dt.round('5min')

# Aggregate data
df_integrated = df_clean.groupby(['timestamp_rounded', 'token_symbol']).agg(agg_dict).reset_index()
df_integrated.rename(columns={'timestamp_rounded': 'timestamp'}, inplace=True)

print(f"   Integrated dataset created")
print(f"   Original records: {len(df_clean)}")
print(f"   Integrated records: {len(df_integrated)}")
print(f"   Reduction: {((len(df_clean) - len(df_integrated))/len(df_clean)*100):.2f}%")

# Add back datetime features
df_integrated['date'] = df_integrated['timestamp'].dt.date
df_integrated['hour'] = df_integrated['timestamp'].dt.hour
df_integrated['day_of_week'] = df_integrated['timestamp'].dt.dayofweek
df_integrated['day_name'] = df_integrated['timestamp'].dt.day_name()

print("\n2. Integrated data sample:")
print(df_integrated.head(10))

In [ ]:
print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80)

df_features = df_integrated.copy()

print("\n1. Creating price-based features...")

# Sort by token and timestamp for proper calculations
df_features = df_features.sort_values(['token_symbol', 'timestamp']).reset_index(drop=True)

# Create lagged features (previous prices)
for token in df_features['token_symbol'].unique():
    mask = df_features['token_symbol'] == token
    
    # Price lags
    df_features.loc[mask, 'price_lag_1'] = df_features.loc[mask, 'price_usd'].shift(1)
    df_features.loc[mask, 'price_lag_2'] = df_features.loc[mask, 'price_usd'].shift(2)
    df_features.loc[mask, 'price_lag_3'] = df_features.loc[mask, 'price_usd'].shift(3)
    
    # Price momentum (rate of change)
    df_features.loc[mask, 'price_momentum'] = df_features.loc[mask, 'price_usd'].pct_change()
    
    # Rolling statistics (3-period window)
    df_features.loc[mask, 'price_rolling_mean_3'] = df_features.loc[mask, 'price_usd'].rolling(window=3, min_periods=1).mean()
    df_features.loc[mask, 'price_rolling_std_3'] = df_features.loc[mask, 'price_usd'].rolling(window=3, min_periods=1).std()
    
    # Volume momentum
    if 'volume_24h' in df_features.columns:
        df_features.loc[mask, 'volume_change'] = df_features.loc[mask, 'volume_24h'].pct_change()

print("Price-based features created")

print("\n2. Creating technical indicators...")

# Volatility measure
df_features['price_volatility'] = df_features.groupby('token_symbol')['price_usd'].transform(
    lambda x: x.rolling(window=5, min_periods=1).std()
)

# Price range
df_features['price_range'] = df_features.groupby('token_symbol')['price_usd'].transform(
    lambda x: x.rolling(window=5, min_periods=1).max() - x.rolling(window=5, min_periods=1).min()
)

print("Technical indicators created")

print("\n3. Creating target variable...")

# Target: Next period's price change (for classification: up/down)
for token in df_features['token_symbol'].unique():
    mask = df_features['token_symbol'] == token
    df_features.loc[mask, 'next_price'] = df_features.loc[mask, 'price_usd'].shift(-1)
    df_features.loc[mask, 'price_change_pct'] = ((df_features.loc[mask, 'next_price'] - df_features.loc[mask, 'price_usd']) 
                                                   / df_features.loc[mask, 'price_usd'] * 100)

# Binary target: 1 if price goes up, 0 if down
df_features['target_direction'] = (df_features['price_change_pct'] > 0).astype(int)

# Multi-class target: significant up (>2%), moderate up (0-2%), moderate down (0 to -2%), significant down (<-2%)
df_features['target_class'] = pd.cut(
    df_features['price_change_pct'],
    bins=[-np.inf, -2, 0, 2, np.inf],
    labels=['significant_down', 'moderate_down', 'moderate_up', 'significant_up']
)

print("   Target variables created")
print(f"   - next_price: Continuous target")
print(f"   - price_change_pct: Percentage change")
print(f"   - target_direction: Binary (0=down, 1=up)")
print(f"   - target_class: 4 classes")

print("\n4. Target distribution:")
print(df_features['target_direction'].value_counts())
print("\nTarget class distribution:")
print(df_features['target_class'].value_counts())

print(f"\nFeature engineering complete")
print(f"Total features: {df_features.shape[1]}")

In [ ]:
print("\n" + "="*80)
print("HANDLING MISSING VALUES & OUTLIERS")
print("="*80)

df_processed = df_features.copy()

print("\n1. Missing Values Analysis:")
missing_summary = pd.DataFrame({
    'Column': df_processed.columns,
    'Missing_Count': df_processed.isnull().sum(),
    'Missing_Percentage': (df_processed.isnull().sum() / len(df_processed) * 100).round(2)
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print(missing_summary)

print("\n2. Handling missing values...")

# For numeric columns, use forward fill then backward fill (appropriate for time series)
numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    if df_processed[col].isnull().sum() > 0:
        df_processed[col] = df_processed.groupby('token_symbol')[col].fillna(method='ffill')
        df_processed[col] = df_processed.groupby('token_symbol')[col].fillna(method='bfill')
        # If still missing, fill with median
        if df_processed[col].isnull().sum() > 0:
            df_processed[col].fillna(df_processed[col].median(), inplace=True)

print("   Missing values handled")
print(f"   Remaining missing values: {df_processed.isnull().sum().sum()}")

print("\n3. Outlier Detection using IQR method...")

# Detect outliers for key numeric columns
outlier_columns = ['price_usd', 'volume_24h', 'market_cap', 'price_change_pct']
outlier_summary = []

for col in outlier_columns:
    if col in df_processed.columns:
        Q1 = df_processed[col].quantile(0.25)
        Q3 = df_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        
        outliers = ((df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)).sum()
        outlier_pct = (outliers / len(df_processed)) * 100
        
        outlier_summary.append({
            'Column': col,
            'Outliers': outliers,
            'Percentage': f"{outlier_pct:.2f}%",
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound
        })
        
        # Cap outliers (winsorization) instead of removing
        df_processed[col] = np.clip(df_processed[col], lower_bound, upper_bound)

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df)
print("\nOutliers capped using winsorization (3*IQR)")

# Remove rows with missing target (last few rows due to shift)
df_processed = df_processed.dropna(subset=['next_price', 'target_direction'])

print(f"\nFinal processed dataset shape: {df_processed.shape}")

In [ ]:
print("\n" + "="*80)
print("DATA STANDARDIZATION & NORMALIZATION")
print("="*80)

# Separate features and targets
feature_cols = [col for col in df_processed.columns if col not in 
                ['timestamp', 'source', 'token_symbol', 'token_name', 'date', 'day_name',
                 'next_price', 'price_change_pct', 'target_direction', 'target_class']]

print(f"\nFeatures to standardize: {len(feature_cols)}")
print(feature_cols)

# Create a copy for standardization
df_standardized = df_processed.copy()

print("\n1. Applying Z-Score Standardization (StandardScaler)...")
scaler_standard = StandardScaler()
df_standardized[feature_cols] = scaler_standard.fit_transform(df_processed[feature_cols])
print("   Z-Score standardization complete")
print(f"   Mean: {df_standardized[feature_cols].mean().mean():.6f}")
print(f"   Std: {df_standardized[feature_cols].std().mean():.6f}")

print("\n2. Creating MinMax normalized version (0-1 scale)...")
df_normalized = df_processed.copy()
scaler_minmax = MinMaxScaler()
df_normalized[feature_cols] = scaler_minmax.fit_transform(df_processed[feature_cols])
print("   MinMax normalization complete")
print(f"   Min: {df_normalized[feature_cols].min().min():.6f}")
print(f"   Max: {df_normalized[feature_cols].max().max():.6f}")

print("\n3. Creating Robust Scaler version (resistant to outliers)...")
df_robust = df_processed.copy()
scaler_robust = RobustScaler()
df_robust[feature_cols] = scaler_robust.fit_transform(df_processed[feature_cols])
print(" Robust scaling complete")

print("\n4. Comparison of scaling methods on sample feature (price_usd):")
comparison_df = pd.DataFrame({
    'Original': df_processed['price_usd'].head(10),
    'StandardScaler': df_standardized['price_usd'].head(10),
    'MinMaxScaler': df_normalized['price_usd'].head(10),
    'RobustScaler': df_robust['price_usd'].head(10)
})
print(comparison_df)

print("\nAll scaling methods applied successfully!")
print("\nRecommendation: Use StandardScaler for most ML models (Random Forest, XGBoost)")
print("               Use MinMaxScaler for neural networks")
print("               Use RobustScaler if outliers are still a concern")

# We'll use StandardScaler version for further analysis
df_final = df_standardized.copy()

In [ ]:
print("\n" + "="*80)
print("EXPLORATORY DATA ANALYSIS (EDA)")
print("="*80)

print("\n1. DATASET OVERVIEW")
print("-" * 80)
print(f"Total Records: {len(df_final)}")
print(f"Total Features: {len(feature_cols)}")
print(f"Date Range: {df_final['timestamp'].min()} to {df_final['timestamp'].max()}")
print(f"Total Trading Days: {df_final['date'].nunique()}")
print(f"Tokens Analyzed: {df_final['token_symbol'].nunique()}")
print(f"Token List: {', '.join(df_final['token_symbol'].unique())}")

print("\n2. TARGET VARIABLE ANALYSIS")
print("-" * 80)
print("\nBinary Target (Price Direction):")
target_counts = df_final['target_direction'].value_counts()
print(f"Up (1): {target_counts.get(1, 0)} ({target_counts.get(1, 0)/len(df_final)*100:.2f}%)")
print(f"Down (0): {target_counts.get(0, 0)} ({target_counts.get(0, 0)/len(df_final)*100:.2f}%)")

print("\nMulti-class Target Distribution:")
print(df_final['target_class'].value_counts())

print("\n3. TOKEN-WISE STATISTICS")
print("-" * 80)
token_stats = df_final.groupby('token_symbol').agg({
    'price_usd': ['mean', 'std', 'min', 'max'],
    'volume_24h': 'mean',
    'market_cap': 'mean',
    'target_direction': lambda x: (x == 1).sum() / len(x) * 100  # % times price went up
}).round(4)
token_stats.columns = ['Price_Mean', 'Price_Std', 'Price_Min', 'Price_Max', 
                       'Avg_Volume', 'Avg_MarketCap', 'Win_Rate_%']
print(token_stats)

print("\n4. CORRELATION ANALYSIS")
print("-" * 80)

# Select numeric features for correlation
numeric_features = df_final[feature_cols].select_dtypes(include=[np.number])
correlation_matrix = numeric_features.corr()

print("\nTop 10 Positive Correlations with Price USD:")
price_corr = correlation_matrix['price_usd'].sort_values(ascending=False)[1:11]
print(price_corr)

print("\nTop 10 Negative Correlations with Price USD:")
price_corr_neg = correlation_matrix['price_usd'].sort_values()[0:10]
print(price_corr_neg)

print("\n5. FEATURE IMPORTANCE (Correlation with Target)")
print("-" * 80)
# Get correlations with binary target
target_correlations = df_final[feature_cols].corrwith(df_final['target_direction']).abs().sort_values(ascending=False)
print("\nTop 15 Features Correlated with Price Direction:")
print(target_correlations.head(15))

print("\n6. TIME-BASED ANALYSIS")
print("-" * 80)
print("\nPrice movement by day of week:")
day_analysis = df_final.groupby('day_name')['target_direction'].agg(['mean', 'count'])
day_analysis.columns = ['Win_Rate', 'Count']
day_analysis['Win_Rate'] = (day_analysis['Win_Rate'] * 100).round(2)
print(day_analysis)

print("\nPrice movement by hour:")
hour_analysis = df_final.groupby('hour')['target_direction'].agg(['mean', 'count'])
hour_analysis.columns = ['Win_Rate', 'Count']
hour_analysis['Win_Rate'] = (hour_analysis['Win_Rate'] * 100).round(2)
print(hour_analysis.head(10))

print("\n7. VOLATILITY ANALYSIS")
print("-" * 80)
volatility_stats = df_final.groupby('token_symbol')['price_volatility'].agg(['mean', 'std', 'min', 'max'])
print(volatility_stats)

print("\n8. MISSING DATA FINAL CHECK")
print("-" * 80)
print(f"Total missing values in final dataset: {df_final.isnull().sum().sum()}")

print("\n" + "="*80)
print("EDA COMPLETE - Ready for Visualization!")
print("="*80)

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 1: PRICE TRENDS OVER TIME")
print("="*80)

# Go back to non-standardized data for visualization
df_viz = df_processed.copy()

fig, axes = plt.subplots(3, 3, figsize=(20, 12))
fig.suptitle('Price Trends by Token (USD)', fontsize=16, fontweight='bold')

tokens = df_viz['token_symbol'].unique()
for idx, token in enumerate(tokens):
    if idx < 9:  # Plot up to 9 tokens
        ax = axes[idx // 3, idx % 3]
        token_data = df_viz[df_viz['token_symbol'] == token].sort_values('timestamp')
        
        ax.plot(token_data['timestamp'], token_data['price_usd'], linewidth=2, color='blue', alpha=0.7)
        ax.fill_between(token_data['timestamp'], token_data['price_usd'], alpha=0.3)
        ax.set_title(f'{token}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Time')
        ax.set_ylabel('Price (USD)')
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Price trends plotted")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 2: CORRELATION HEATMAP")
print("="*80)

# Select top correlated features for cleaner visualization
top_features = target_correlations.head(15).index.tolist()
top_features.append('target_direction')

correlation_subset = df_final[top_features].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(correlation_subset, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap - Top 15 Features vs Target', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("Correlation heatmap plotted")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 3: FEATURE DISTRIBUTIONS")
print("="*80)

# Plot distributions of key features
key_features = ['price_usd', 'volume_24h', 'market_cap', 'price_change_24h', 
                'price_volatility', 'price_momentum']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Distribution of Key Features', fontsize=16, fontweight='bold')

for idx, feature in enumerate(key_features):
    if feature in df_viz.columns:
        ax = axes[idx // 3, idx % 3]
        
        # Histogram with KDE
        df_viz[feature].hist(bins=50, ax=ax, alpha=0.6, color='skyblue', edgecolor='black')
        ax2 = ax.twinx()
        df_viz[feature].plot(kind='kde', ax=ax2, color='red', linewidth=2)
        
        ax.set_title(f'{feature}', fontsize=12, fontweight='bold')
        ax.set_xlabel(feature)
        ax.set_ylabel('Frequency', color='blue')
        ax2.set_ylabel('Density', color='red')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Distribution plots created")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 4: BOX PLOTS FOR OUTLIER DETECTION")
print("="*80)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Box Plots - Outlier Detection', fontsize=16, fontweight='bold')

for idx, feature in enumerate(key_features):
    if feature in df_viz.columns:
        ax = axes[idx // 3, idx % 3]
        df_viz.boxplot(column=feature, by='token_symbol', ax=ax, rot=90)
        ax.set_title(f'{feature}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Token')
        ax.set_ylabel(feature)
        plt.sca(ax)
        plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

print("Box plots created")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 5: TARGET VARIABLE ANALYSIS")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Target Variable Analysis', fontsize=16, fontweight='bold')

# 1. Binary target distribution
ax1 = axes[0, 0]
target_counts = df_viz['target_direction'].value_counts()
colors = ['#ff6b6b', '#51cf66']
ax1.bar(target_counts.index, target_counts.values, color=colors, alpha=0.7, edgecolor='black')
ax1.set_title('Binary Target Distribution (0=Down, 1=Up)', fontweight='bold')
ax1.set_xlabel('Direction')
ax1.set_ylabel('Count')
ax1.set_xticks([0, 1])
ax1.set_xticklabels(['Down', 'Up'])
for i, v in enumerate(target_counts.values):
    ax1.text(i, v + 50, str(v), ha='center', fontweight='bold')

# 2. Multi-class target distribution
ax2 = axes[0, 1]
class_counts = df_viz['target_class'].value_counts()
colors2 = ['#e74c3c', '#f39c12', '#3498db', '#2ecc71']
ax2.bar(range(len(class_counts)), class_counts.values, color=colors2, alpha=0.7, edgecolor='black')
ax2.set_title('Multi-class Target Distribution', fontweight='bold')
ax2.set_xlabel('Class')
ax2.set_ylabel('Count')
ax2.set_xticks(range(len(class_counts)))
ax2.set_xticklabels(class_counts.index, rotation=45, ha='right')
for i, v in enumerate(class_counts.values):
    ax2.text(i, v + 20, str(v), ha='center', fontweight='bold')

# 3. Target distribution by token
ax3 = axes[1, 0]
token_target = df_viz.groupby('token_symbol')['target_direction'].mean() * 100
token_target.sort_values(ascending=False).plot(kind='barh', ax=ax3, color='steelblue', alpha=0.7, edgecolor='black')
ax3.set_title('Win Rate by Token (%)', fontweight='bold')
ax3.set_xlabel('Win Rate (%)')
ax3.set_ylabel('Token')
ax3.axvline(x=50, color='red', linestyle='--', linewidth=2, label='50% baseline')
ax3.legend()

# 4. Price change percentage distribution
ax4 = axes[1, 1]
df_viz['price_change_pct'].hist(bins=50, ax=ax4, alpha=0.7, color='purple', edgecolor='black')
ax4.axvline(x=0, color='red', linestyle='--', linewidth=2, label='No change')
ax4.set_title('Distribution of Price Changes (%)', fontweight='bold')
ax4.set_xlabel('Price Change (%)')
ax4.set_ylabel('Frequency')
ax4.legend()

plt.tight_layout()
plt.show()

print("Target variable analysis plots created")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 6: TIME SERIES PATTERNS")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Time Series Patterns Analysis', fontsize=16, fontweight='bold')

# 1. Average price by day of week
ax1 = axes[0, 0]
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_price = df_viz.groupby('day_name')['price_usd'].mean().reindex(day_order)
day_price.plot(kind='bar', ax=ax1, color='teal', alpha=0.7, edgecolor='black')
ax1.set_title('Average Price by Day of Week', fontweight='bold')
ax1.set_xlabel('Day')
ax1.set_ylabel('Average Price (USD)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# 2. Win rate by day of week
ax2 = axes[0, 1]
day_win = df_viz.groupby('day_name')['target_direction'].mean().reindex(day_order) * 100
day_win.plot(kind='bar', ax=ax2, color='coral', alpha=0.7, edgecolor='black')
ax2.axhline(y=50, color='red', linestyle='--', linewidth=2, label='50% baseline')
ax2.set_title('Win Rate by Day of Week', fontweight='bold')
ax2.set_xlabel('Day')
ax2.set_ylabel('Win Rate (%)')
ax2.tick_params(axis='x', rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# 3. Average price by hour
ax3 = axes[1, 0]
hour_price = df_viz.groupby('hour')['price_usd'].mean()
ax3.plot(hour_price.index, hour_price.values, marker='o', linewidth=2, markersize=8, color='darkblue')
ax3.fill_between(hour_price.index, hour_price.values, alpha=0.3)
ax3.set_title('Average Price by Hour of Day', fontweight='bold')
ax3.set_xlabel('Hour (24h format)')
ax3.set_ylabel('Average Price (USD)')
ax3.grid(True, alpha=0.3)
ax3.set_xticks(range(0, 24, 2))

# 4. Trading volume over time
ax4 = axes[1, 1]
volume_by_token = df_viz.groupby(['timestamp', 'token_symbol'])['volume_24h'].sum().reset_index()
for token in volume_by_token['token_symbol'].unique()[:5]:  # Top 5 tokens
    token_vol = volume_by_token[volume_by_token['token_symbol'] == token]
    ax4.plot(token_vol['timestamp'], token_vol['volume_24h'], label=token, linewidth=2, alpha=0.7)
ax4.set_title('24h Trading Volume by Token', fontweight='bold')
ax4.set_xlabel('Time')
ax4.set_ylabel('Volume (USD)')
ax4.legend(loc='best')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Time series pattern plots created")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 7: ADVANCED ANALYSIS")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Advanced Analysis Plots', fontsize=16, fontweight='bold')

# 1. Volatility vs Returns scatter
ax1 = axes[0, 0]
for token in df_viz['token_symbol'].unique()[:7]:
    token_data = df_viz[df_viz['token_symbol'] == token]
    ax1.scatter(token_data['price_volatility'], token_data['price_change_pct'], 
               label=token, alpha=0.6, s=50)
ax1.set_title('Volatility vs Price Change', fontweight='bold')
ax1.set_xlabel('Price Volatility')
ax1.set_ylabel('Price Change (%)')
ax1.legend(loc='best', fontsize=8)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='red', linestyle='--', linewidth=1)

# 2. Volume vs Price correlation
ax2 = axes[0, 1]
for token in df_viz['token_symbol'].unique()[:7]:
    token_data = df_viz[df_viz['token_symbol'] == token]
    ax2.scatter(token_data['volume_24h'], token_data['price_usd'], 
               label=token, alpha=0.6, s=50)
ax2.set_title('Volume vs Price Relationship', fontweight='bold')
ax2.set_xlabel('24h Volume (USD)')
ax2.set_ylabel('Price (USD)')
ax2.legend(loc='best', fontsize=8)
ax2.grid(True, alpha=0.3)

# 3. Rolling mean comparison
ax3 = axes[1, 0]
sample_token = df_viz['token_symbol'].unique()[0]  # Pick first token
sample_data = df_viz[df_viz['token_symbol'] == sample_token].sort_values('timestamp')
ax3.plot(sample_data['timestamp'], sample_data['price_usd'], label='Actual Price', linewidth=2, alpha=0.7)
ax3.plot(sample_data['timestamp'], sample_data['price_rolling_mean_3'], 
        label='3-Period Rolling Mean', linewidth=2, linestyle='--')
ax3.set_title(f'Price vs Rolling Mean - {sample_token}', fontweight='bold')
ax3.set_xlabel('Time')
ax3.set_ylabel('Price (USD)')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# 4. Market cap distribution
ax4 = axes[1, 1]
market_cap_by_token = df_viz.groupby('token_symbol')['market_cap'].mean().sort_values(ascending=False)
colors_map = plt.cm.viridis(np.linspace(0, 1, len(market_cap_by_token)))
ax4.barh(range(len(market_cap_by_token)), market_cap_by_token.values, color=colors_map, edgecolor='black')
ax4.set_yticks(range(len(market_cap_by_token)))
ax4.set_yticklabels(market_cap_by_token.index)
ax4.set_title('Average Market Cap by Token', fontweight='bold')
ax4.set_xlabel('Market Cap (USD)')
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("Advanced analysis plots created")

In [ ]:
print("\n" + "="*80)
print("VISUALIZATION 8: FEATURE IMPORTANCE PREVIEW")
print("="*80)

# Plot top features correlated with target
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# 1. Top positive correlations
ax1 = axes[0]
top_positive = target_correlations.head(15)
top_positive.plot(kind='barh', ax=ax1, color='green', alpha=0.7, edgecolor='black')
ax1.set_title('Top 15 Features - Correlation with Target', fontweight='bold', fontsize=14)
ax1.set_xlabel('Absolute Correlation')
ax1.set_ylabel('Feature')
ax1.grid(True, alpha=0.3, axis='x')

# 2. Correlation strength distribution
ax2 = axes[1]
all_correlations = df_final[feature_cols].corrwith(df_final['target_direction']).abs()
ax2.hist(all_correlations, bins=30, alpha=0.7, color='blue', edgecolor='black')
ax2.axvline(x=all_correlations.mean(), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {all_correlations.mean():.3f}')
ax2.set_title('Distribution of Feature Correlations', fontweight='bold', fontsize=14)
ax2.set_xlabel('Absolute Correlation with Target')
ax2.set_ylabel('Number of Features')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Feature importance visualization created")

In [ ]:
print("\n" + "="*80)
print("FINAL DATASET SUMMARY & PREPARATION FOR ML")
print("="*80)

print("\n1. FINAL DATASET STATISTICS")
print("-" * 80)
print(f"Total Records: {len(df_final)}")
print(f"Total Features: {len(feature_cols)}")
print(f"Training-ready samples: {len(df_final.dropna())}")
print(f"Missing values: {df_final.isnull().sum().sum()}")

print("\n2. FEATURE CATEGORIES")
print("-" * 80)
price_features = [f for f in feature_cols if 'price' in f.lower()]
volume_features = [f for f in feature_cols if 'volume' in f.lower()]
time_features = [f for f in feature_cols if any(x in f.lower() for x in ['hour', 'day', 'week'])]
technical_features = [f for f in feature_cols if any(x in f.lower() for x in ['volatility', 'momentum', 'rolling'])]

print(f"Price-related features: {len(price_features)}")
print(f"Volume-related features: {len(volume_features)}")
print(f"Time-based features: {len(time_features)}")
print(f"Technical indicators: {len(technical_features)}")

print("\n3. TARGET VARIABLE SUMMARY")
print("-" * 80)
print(f"Binary Target (target_direction):")
print(f"  - Class 0 (Down): {(df_final['target_direction'] == 0).sum()}")
print(f"  - Class 1 (Up): {(df_final['target_direction'] == 1).sum()}")
print(f"  - Balance ratio: {min(target_counts) / max(target_counts):.2f}")

print("\n4. RECOMMENDED ML MODELS")
print("-" * 80)
print("Based on the data characteristics:")
print("  Random Forest Classifier - Handles non-linear relationships well")
print("  XGBoost/LightGBM - Excellent for tabular data, fast training")
print("  Gradient Boosting - Good performance on imbalanced data")
print("  Neural Networks - May need more data, longer training")
print("  Logistic Regression - Simple baseline, may underfit")

print("\n5. DATA QUALITY CHECKS")
print("-" * 80)
print("No missing values in features")
print("No duplicate records")
print("Outliers handled via winsorization")
print("Features standardized (Z-score)")
print("Target variable balanced enough for training")

print("\n6. TRAIN/TEST SPLIT RECOMMENDATION")
print("-" * 80)
print("Recommended split strategy:")
print("  - Time-based split (80% train, 20% test)")
print("  - Cross-validation: 5-fold TimeSeriesSplit")
print("  - Avoid data leakage by respecting temporal order")

print("\n7. EXPORTING PREPARED DATASETS")
print("-" * 80)

# Export different versions
output_files = {
    'processed': df_processed,      # Non-standardized
    'standardized': df_final,       # Z-score standardized
    'normalized': df_normalized,    # MinMax scaled
    'robust': df_robust            # Robust scaled
}

for name, data in output_files.items():
    filename = f'./crypto_data_{name}.csv'
    data.to_csv(filename, index=False)
    print(f"Saved: {filename} ({len(data)} rows, {len(data.columns)} columns)")

print("\n8. FEATURE LIST FOR ML TRAINING")
print("-" * 80)
print("Copy this list for your ML model:")
print(f"FEATURE_COLS = {feature_cols}")

print("\n9. READY FOR MODEL TRAINING")
print("-" * 80)
print("Next steps:")
print("  1. Split data using TimeSeriesSplit")
print("  2. Train Random Forest / XGBoost models")
print("  3. Evaluate using SMAPE, MAE, Accuracy, F1-Score")
print("  4. Perform hyperparameter tuning (5-fold CV)")
print("  5. Select best model for deployment")

print("\n" + "="*80)
print("DATA PROCESSING & EDA COMPLETE!")
print("="*80)
print("\nSummary:")
print(f"   - Collected data from 2 sources (CoinGecko + CoinMarketCap)")
print(f"   - Cleaned and processed {len(df_final)} records")
print(f"   - Created {len(feature_cols)} features")
print(f"   - Generated comprehensive visualizations")
print(f"   - Prepared 4 different scaled versions")
print(f"   - Ready for ML model training!")

In [ ]:
print("\n" + "="*80)
print("ML PREPROCESSING HELPER CODE (Use before training)")
print("="*80)

# This cell prepares X and y for ML training
# Use the standardized dataset
X = df_final[feature_cols].copy()
y_binary = df_final['target_direction'].copy()
y_multiclass = df_final['target_class'].copy()
y_regression = df_final['price_change_pct'].copy()

print(f"Features (X) shape: {X.shape}")
print(f"Binary target (y_binary) shape: {y_binary.shape}")
print(f"Multiclass target (y_multiclass) shape: {y_multiclass.shape}")
print(f"Regression target (y_regression) shape: {y_regression.shape}")

print("\n✅ X and y variables ready for model training")
print("\nQuick verification:")
print(f"  - No NaN in X: {X.isnull().sum().sum() == 0}")
print(f"  - No NaN in y_binary: {y_binary.isnull().sum() == 0}")
print(f"  - Feature types: {X.dtypes.value_counts().to_dict()}")

print("\n" + "="*80)
print("END OF DATA PROCESSING & VISUALIZATION")
print("="*80)